<a href="https://colab.research.google.com/github/JumanaRahim/Nullclass-Internship/blob/main/TASK_2%264.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install opencv-python numpy torch torchvision tkinter


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [8]:
class ColorizationAutoencoder(nn.Module):
    def __init__(self):
        super(ColorizationAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()  # Output in range [-1, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [9]:
# Define Data Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

# Load CIFAR-10 Dataset (Small Subset for Quick Training)
trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
subset_size = 5000
train_subset = torch.utils.data.Subset(trainset, range(subset_size))
trainloader = DataLoader(train_subset, batch_size=64, shuffle=True)



Files already downloaded and verified


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Define Model
class ColorizationAutoencoder(nn.Module):
    def __init__(self):
        super(ColorizationAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()  # Output in range [-1, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# ✅ Initialize Model, Loss Function & Optimizer
model = ColorizationAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# ✅ Ensure num_epochs is defined
num_epochs = 3

# ✅ Fix Grayscale to RGB Conversion
def convert_grayscale_to_rgb(grayscale_tensor):
    """Convert grayscale image (1 channel) to 3-channel grayscale image"""
    return grayscale_tensor.repeat(1, 3, 1, 1)

# ✅ Training Loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for gray_imgs, color_imgs in trainloader:
        gray_imgs, color_imgs = gray_imgs.to(device), color_imgs.to(device)

        # ✅ Convert grayscale to 3-channel grayscale
        color_imgs = convert_grayscale_to_rgb(color_imgs)

        optimizer.zero_grad()  # ✅ Now optimizer is defined
        outputs = model(gray_imgs)
        loss = criterion(outputs, color_imgs)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(trainloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Modify GradScaler (works for CPU & GPU)
if device.type == "cuda":
    scaler = torch.amp.GradScaler("cuda")
else:
    scaler = None  # AMP not needed for CPU

# ✅ Modify AMP usage in training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for gray_imgs, color_imgs in trainloader:
        gray_imgs, color_imgs = gray_imgs.to(device), color_imgs.to(device)

        optimizer.zero_grad()

        if scaler:  # ✅ Use AMP only if CUDA is available
            with torch.amp.autocast("cuda"):
                outputs = model(gray_imgs)
                loss = criterion(outputs, color_imgs)
        else:
            outputs = model(gray_imgs)
            loss = criterion(outputs, color_imgs)

        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        total_loss += loss.item()



In [ ]:
torch.save(model.state_dict(), "colorization_model.pth")


In [ ]:
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image

# Load Trained Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ColorizationAutoencoder().to(device)
model.load_state_dict(torch.load("colorization_model.pth", map_location=device))
model.eval()

# Define Transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

def colorize_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_3ch = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    img_tensor = transform(gray_3ch).unsqueeze(0).to(device)

    with torch.no_grad():
        colorized_tensor = model(img_tensor)

    colorized_img = colorized_tensor.squeeze(0).cpu().numpy().transpose(1, 2, 0)
    colorized_img = (colorized_img * 255).astype(np.uint8)

    return colorized_img

# Open Webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    colorized_frame = colorize_frame(frame)

    cv2.imshow("Colorized Video", colorized_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [ ]:
import tkinter as tk
from tkinter import filedialog
import subprocess

def start_webcam():
    subprocess.run(["python", "video_colorization.py"])

root = tk.Tk()
root.title("Real-Time Video Colorization")
root.geometry("300x200")

btn_webcam = tk.Button(root, text="Start Webcam Colorization", command=start_webcam)
btn_webcam.pack(pady=20)

root.mainloop()




In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import numpy as np

def evaluate_model():
    all_preds = []
    all_labels = []

    for gray_imgs, color_imgs in trainloader:
        gray_imgs, color_imgs = gray_imgs.to(device), color_imgs.to(device)

        with torch.no_grad():
            outputs = model(gray_imgs)

        pred_classes = (outputs * 255).byte().cpu().numpy().flatten()
        true_classes = (color_imgs * 255).byte().cpu().numpy().flatten()

        all_preds.extend(pred_classes)
        all_labels.extend(true_classes)

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy: {accuracy * 100:.2f}%")

evaluate_model()

